In [1]:
# Part 1

In [2]:
with open("data.txt") as file:
    inputs = [input_.strip() for input_ in file.readlines()]

In [3]:
rules = """departure location: 31-538 or 546-960
departure station: 39-660 or 673-960
departure platform: 35-731 or 745-968
departure track: 43-179 or 185-953
departure date: 29-250 or 263-949
departure time: 43-903 or 928-954
arrival location: 46-372 or 384-968
arrival station: 36-215 or 225-950
arrival platform: 25-631 or 655-950
arrival track: 26-768 or 781-962
class: 29-462 or 478-974
duration: 34-441 or 455-963
price: 39-683 or 693-956
route: 36-342 or 348-971
row: 37-501 or 520-963
seat: 46-356 or 369-973
train: 43-414 or 423-954
type: 35-160 or 178-950
wagon: 29-878 or 889-959
zone: 31-188 or 201-971"""

In [4]:
valid_numbers = []
for rule in rules.split("\n"):
    valid_val = rule.split(": ")[1].split(" or ")
    valid_numbers.extend(valid_val)

In [5]:
def is_valid(val):
    for valid in valid_numbers:
            min_, max_ = tuple(map(int, valid.split("-")))
            if val in range(min_, max_):
                return True
    return False

In [6]:
invalid = 0
for input_ in inputs:
    values = list(map(int, input_.split(",")))
    for val in values:
        if not is_valid(val):
            invalid += val

In [7]:
# part 2

# I'm excited to use CSPs in a real problem for the first time! 

In [8]:
# get rid of invalid tickets
pruned_input = []
for input_ in inputs:
    values = list(map(int, input_.split(",")))
    is_valid_ticket = True
    for val in values:
        if not is_valid(val):
            is_valid_ticket = False
    if is_valid_ticket:
        pruned_input.append(list(map(int, input_.split(","))))

In [9]:
my_ticket = [137,149,139,127,83,61,89,53,73,67,131,113,109,101,71,59,103,97,107,79]
pruned_input.append(my_ticket)

In [10]:
rule_dict = {rule.split(": ")[0]: rule.split(": ")[1] for rule in rules.split("\n")}

In [11]:
for rule in rule_dict: 
    range_1, range_2 = rule_dict[rule].split(" or ")
    range_1 = range(*[int(range_) + i for i, range_ in enumerate(range_1.split("-"))])
    range_2 = range(*[int(range_) + i for i, range_ in enumerate(range_2.split("-"))])
    rule_dict[rule] = range_1, range_2

In [12]:
def is_possible(rule, pos):
    range_1, range_2 = rule_dict[rule]
    for ticket in pruned_input:
        val = ticket[pos]
        if val not in range_1 and val not in range_2:
            return False
    return True

In [13]:
pos_to_rule = {}
for pos in range(len(rule_dict.keys())):
    possible_rules = []
    for rule in rule_dict:
        if is_possible(rule, pos):
            possible_rules.append(rule)
    pos_to_rule[pos] = frozenset(possible_rules)
        

In [14]:
from constraint import *

In [15]:
problem = Problem()
for pos in pos_to_rule:
    problem.addVariable(pos, list(pos_to_rule[pos]))
problem.addConstraint(AllDifferentConstraint())
sol = {rule: pos for pos, rule in problem.getSolutions()[0].items()}

In [16]:
sol

{'wagon': 12,
 'route': 13,
 'class': 0,
 'row': 9,
 'departure platform': 16,
 'departure track': 1,
 'departure station': 14,
 'departure date': 17,
 'departure location': 2,
 'departure time': 6,
 'type': 10,
 'train': 11,
 'zone': 5,
 'duration': 19,
 'arrival station': 4,
 'price': 7,
 'arrival location': 18,
 'seat': 8,
 'arrival platform': 3,
 'arrival track': 15}

In [17]:
my_ticket[16] * my_ticket[1] * my_ticket[14] * my_ticket[17] * my_ticket[2] * my_ticket[6]

1307550234719